<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/philipp/notebooks/Dashboard_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q&A Dasbboard / Live Demo

In [1]:
%%capture
!pip install gradio
# !pip install git+https://github.com/openai/whisper.git
# import whisper
import gradio as gr
import pandas as pd

In [2]:
dfs = list()

for questionnaire in range(1, 6):
    url = f'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/questionnaires/questionnaire{questionnaire}.json'
    df = pd.read_json(url)
    df['questionnaire'] = questionnaire
    df = df[['questionnaire', 'question']]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

questionnaire_names = [f"Questionnaire {i}" for i in range(1, 6)]
answered_questions = list()

In [3]:
def chatbot(selected_option, input_message, chat_history):
    if not chat_history and input_message != "start":
        response = "Please select a questionnaire and start the conversation with the keyword 'start'."
        chat_history.append((input_message, response))
        return chat_history, chat_history

    all_questions = df[df['questionnaire'] == int(selected_option[-1])]['question'].tolist()
    unanswered_questions = [q for q in all_questions if q not in answered_questions]
    unanswered_questions_string = '\n'.join(unanswered_questions)

    response = "Please answer the following questions:\n" + unanswered_questions_string
    chat_history.append((input_message, response))
    return chat_history, chat_history


with gr.Blocks() as app:
    gr.Markdown("# Questionnaire Bot")

    option = gr.Dropdown(
        questionnaire_names,
        label="Choose a questionnaire to answer"
    )

    chatbot_interface = gr.Chatbot(label="Interactive Chat")

    message = gr.Textbox(label="Your answers", placeholder="Answer the question ...")

    send_button = gr.Button("Send")

    chat_history = gr.State([])

    send_button.click(
        fn=chatbot,
        inputs=[option, message, chat_history],
        outputs=[chatbot_interface, chat_history]
    )

app.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6e8a4d07d282a63f66.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6e8a4d07d282a63f66.gradio.live
